In [16]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet, ResNet50, VGG16, Xception
from sklearn.model_selection import train_test_split
import autokeras as ak
import netron

In [18]:
df=pd.read_csv('train.csv')

In [19]:
df

,filepath,team_name,team
0,../input/english-premier-league-logo-detection...,manchester-united,12
1,../input/english-premier-league-logo-detection...,manchester-united,12
2,../input/english-premier-league-logo-detection...,manchester-united,12
3,../input/english-premier-league-logo-detection...,manchester-united,12
4,../input/english-premier-league-logo-detection...,manchester-united,12
...,...,...,...
19995,../input/english-premier-league-logo-detection...,brentford,2
19996,../input/english-premier-league-logo-detection...,brentford,2
19997,../input/english-premier-league-logo-detection...,brentford,2
19998,../input/english-premier-league-logo-detection...,brentford,2


In [ ]:
df['filepath'] = df['filepath'].str.replace(
    '../input/english-premier-league-logo-detection-20k-images/', 
    "{Path to folder containing the dataset}"
)

# Extracting Images

In [51]:
def extract_image(filepath):
    img = cv2.imread(filepath)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        print(f"Error reading image at {filepath}")
    return img

df['image'] = df['filepath'].apply(extract_image)


In [52]:
df['image'].apply(lambda x: x.shape).value_counts()

(140, 140, 3)    17015
(112, 112, 3)     2985
Name: image, dtype: int64

In [53]:
df['image'] = df['image'].apply(lambda x: cv2.resize(x, (140,140)))
df['image'].apply(lambda x: x.shape).value_counts()

(140, 140, 3)    20000
Name: image, dtype: int64

In [54]:
df['image'] = df['image'].apply(lambda x: x/255)
df['image'].apply(lambda x: x.shape).value_counts()


(140, 140, 3)    20000
Name: image, dtype: int64

In [55]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['team'], random_state=42)

train_df['team'].value_counts()

16    800
15    800
14    800
5     800
7     800
1     800
17    800
13    800
4     800
12    800
2     800
19    800
3     800
0     800
8     800
6     800
10    800
9     800
11    800
18    800
Name: team, dtype: int64

In [56]:
X_train = np.array(train_df['image'].tolist())
y_train =  pd.get_dummies(train_df['team_name']).values
X_test = np.array(test_df['image'].tolist())
y_test = pd.get_dummies(test_df['team_name']).values

# Resnet Model 

In [19]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(140, 140, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

94765736/94765736 [==============================] - 7s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 140, 140, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 146, 146, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 70, 70, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 70, 70, 64)           256       ['conv1_conv[0][0]']          
 on)                         

In [32]:
resnet_history = model.fit(X_train,  y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
400/400 [==============================] - 370s 923ms/step - loss: 0.0495 - accuracy: 0.9840 - val_loss: 0.0352 - val_accuracy: 0.9897
Epoch 2/5
400/400 [==============================] - 333s 833ms/step - loss: 0.0387 - accuracy: 0.9879 - val_loss: 0.0748 - val_accuracy: 0.9766
Epoch 3/5
400/400 [==============================] - 342s 856ms/step - loss: 0.0410 - accuracy: 0.9871 - val_loss: 0.0288 - val_accuracy: 0.9894
Epoch 4/5
400/400 [==============================] - 342s 855ms/step - loss: 0.0447 - accuracy: 0.9884 - val_loss: 0.0479 - val_accuracy: 0.9834
Epoch 5/5
400/400 [==============================] - 342s 855ms/step - loss: 0.0276 - accuracy: 0.9913 - val_loss: 0.0418 - val_accuracy: 0.9853


In [33]:
with open('resnet_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [34]:
with open('resnet_history.pkl', 'wb') as file:
    pickle.dump(resnet_history.history, file)

# Xception Model 

In [43]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(140, 140, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)

xception_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
xception_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 140, 140, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 69, 69, 32)           864       ['input_2[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 69, 69, 32)           128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 69, 69, 32)           0         ['block1_conv1_bn[0][0]'

In [44]:
xception_hist = xception_model.fit(X_train,  y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
200/200 [==============================] - 355s 2s/step - loss: 0.0747 - accuracy: 0.9820 - val_loss: 0.0215 - val_accuracy: 0.9941
Epoch 2/5
200/200 [==============================] - 349s 2s/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0017 - val_accuracy: 0.9997
Epoch 3/5
200/200 [==============================] - 338s 2s/step - loss: 2.4635e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 0.9997
Epoch 4/5
200/200 [==============================] - 331s 2s/step - loss: 1.5516e-04 - accuracy: 1.0000 - val_loss: 9.7623e-04 - val_accuracy: 0.9997
Epoch 5/5
200/200 [==============================] - 354s 2s/step - loss: 1.0837e-04 - accuracy: 1.0000 - val_loss: 8.1365e-04 - val_accuracy: 0.9997


In [269]:
with open('xception_model.pkl', 'wb') as file:
    pickle.dump(xception_model, file)

In [270]:
with open('xception_history.pkl', 'wb') as file:
    pickle.dump(xception_hist.history, file)

# Custom Model 1

In [38]:
own_model1 = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(140, 140, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(df['team'].nunique(), activation='softmax')
    ])

own_model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

own_model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 138, 138, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 69, 69, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 67, 67, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 33, 33, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 31, 31, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 15, 15, 128)      

In [39]:
own_history = own_model1.fit(X_train,  y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
400/400 [==============================] - 183s 455ms/step - loss: 0.5079 - accuracy: 0.8443 - val_loss: 0.0197 - val_accuracy: 0.9941
Epoch 2/5
400/400 [==============================] - 174s 434ms/step - loss: 0.0740 - accuracy: 0.9779 - val_loss: 0.0071 - val_accuracy: 0.9975
Epoch 3/5
400/400 [==============================] - 181s 454ms/step - loss: 0.0438 - accuracy: 0.9866 - val_loss: 0.0132 - val_accuracy: 0.9959
Epoch 4/5
400/400 [==============================] - 177s 442ms/step - loss: 0.0421 - accuracy: 0.9882 - val_loss: 0.0129 - val_accuracy: 0.9953
Epoch 5/5
400/400 [==============================] - 163s 407ms/step - loss: 0.0160 - accuracy: 0.9947 - val_loss: 0.0106 - val_accuracy: 0.9975


In [40]:
with open('own1_model.pkl', 'wb') as file:
    pickle.dump(own_model1, file)

In [ ]:
with open('own1_history.pkl', 'wb') as file:
    pickle.dump(own_history.history, file)

# MobileNet Model 

In [172]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(140, 140, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)

mobile_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

mobile_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
mobile_model.summary()


17225924/17225924 [==============================] - 3s 0us/step
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 140, 140, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 70, 70, 32)        864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 70, 70, 32)        128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 70, 70, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 70, 70, 32)        288       
 )                                                               
                                                            

In [173]:
mobile_hist = mobile_model.fit(X_train,  y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
200/200 [==============================] - 97s 473ms/step - loss: 0.0718 - accuracy: 0.9812 - val_loss: 0.0023 - val_accuracy: 0.9997
Epoch 2/5
200/200 [==============================] - 92s 461ms/step - loss: 4.3768e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 0.9997
Epoch 3/5
200/200 [==============================] - 87s 437ms/step - loss: 1.6358e-04 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 0.9997
Epoch 4/5
200/200 [==============================] - 88s 440ms/step - loss: 9.1118e-05 - accuracy: 1.0000 - val_loss: 8.9919e-04 - val_accuracy: 0.9997
Epoch 5/5
200/200 [==============================] - 89s 447ms/step - loss: 6.0793e-05 - accuracy: 1.0000 - val_loss: 7.6618e-04 - val_accuracy: 0.9997


In [174]:
with open('mobilenet_model.pkl', 'wb') as file:
    pickle.dump(mobile_model, file)

In [175]:
with open('mobilenet_history.pkl', 'wb') as file:
    pickle.dump(mobile_hist.history, file)

# Custom Model 2

In [178]:
own_model2 = models.Sequential()

own_model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(140, 140, 3)))
own_model2.add(layers.AveragePooling2D((2, 2)))
own_model2.add(layers.Dropout(0.25))

own_model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
own_model2.add(layers.AveragePooling2D((2, 2)))
own_model2.add(layers.Dropout(0.25))

own_model2.add(layers.Flatten())

own_model2.add(layers.Dense(128, activation='relu'))
own_model2.add(layers.Dropout(0.5))

own_model2.add(layers.Dense(20, activation='softmax'))

own_model2.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

own_model2.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 138, 138, 32)      896       
                                                                 
 average_pooling2d_2 (Avera  (None, 69, 69, 32)        0         
 gePooling2D)                                                    
                                                                 
 dropout_5 (Dropout)         (None, 69, 69, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 67, 67, 64)        18496     
                                                                 
 average_pooling2d_3 (Avera  (None, 33, 33, 64)        0         
 gePooling2D)                                                    
                                                                 
 dropout_6 (Dropout)         (None, 33, 33, 64)       

In [179]:
own2_hist = own_model2.fit(X_train,  y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
200/200 [==============================] - 143s 710ms/step - loss: 0.6516 - accuracy: 0.8151 - val_loss: 0.0279 - val_accuracy: 0.9937
Epoch 2/5
200/200 [==============================] - 150s 751ms/step - loss: 0.0808 - accuracy: 0.9762 - val_loss: 0.0141 - val_accuracy: 0.9956
Epoch 3/5
200/200 [==============================] - 154s 771ms/step - loss: 0.0444 - accuracy: 0.9864 - val_loss: 0.0099 - val_accuracy: 0.9956
Epoch 4/5
200/200 [==============================] - 137s 684ms/step - loss: 0.0326 - accuracy: 0.9905 - val_loss: 0.0157 - val_accuracy: 0.9959
Epoch 5/5
200/200 [==============================] - 140s 699ms/step - loss: 0.0310 - accuracy: 0.9895 - val_loss: 0.0045 - val_accuracy: 0.9981


In [180]:
with open('own2_model.pkl', 'wb') as file:
    pickle.dump(own_model2, file)

In [181]:
with open('own2_history.pkl', 'wb') as file:
    pickle.dump(own2_hist.history, file)

# VGG Model 

In [209]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(140, 140, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


58889256/58889256 [==============================] - 5s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 140, 140, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 140, 140, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 140, 140, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 70, 70, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 70, 70, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 70, 70, 128)       147584    
                                                            

In [210]:
history = model.fit(X_train,  y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
400/400 [==============================] - 746s 2s/step - loss: 0.2821 - accuracy: 0.9524 - val_loss: 0.0258 - val_accuracy: 0.9987
Epoch 2/5
400/400 [==============================] - 682s 2s/step - loss: 0.0137 - accuracy: 0.9994 - val_loss: 0.0092 - val_accuracy: 0.9987
Epoch 3/5
400/400 [==============================] - 693s 2s/step - loss: 0.0050 - accuracy: 0.9996 - val_loss: 0.0057 - val_accuracy: 0.9991
Epoch 4/5
400/400 [==============================] - 672s 2s/step - loss: 0.0027 - accuracy: 0.9999 - val_loss: 0.0061 - val_accuracy: 0.9991
Epoch 5/5
400/400 [==============================] - 666s 2s/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.0031 - val_accuracy: 0.9994


In [211]:
with open('vgg_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [212]:
with open('vgg_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

# NAS Model 

In [ ]:
image_classifier = ak.ImageClassifier(max_trials=2) 


nas_hist = image_classifier.fit(X_train, y_train, epochs=2, validation_split=0.2)

loss, accuracy = image_classifier.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

best_model = image_classifier.export_model()
best_model_history = best_model.fit(X_train, y_train, epochs=3, validation_split=0.2)

In [ ]:
import pickle 
with open('nas_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

with open('nas_history.pkl', 'wb') as file:
    pickle.dump(best_model_history.history, file)

#  Visualizing Model Architecture with Netron

In [1]:

for i in ['resnet', 'xception', 'own1', 'mobilenet', 'own2', 'vgg', 'nas']:
    netron.start(f"Models/{i}_model.pkl")


Serving 'Models/nas_model.pkl' at http://localhost:8080


('localhost', 8080)

# Slicing and Merging Model Architecture Image

In [16]:

from PIL import Image

image_path = '{Image Path} '
img = Image.open(image_path)
width, height = img.size
num_slices = 4  # Example: Split into 4 parts
slice_height = height // num_slices
slices = [img.crop((0, i * slice_height, width, (i + 1) * slice_height)) for i in range(num_slices)]
new_width = width * num_slices 
combined_image = Image.new("RGB", (new_width, slice_height))
for i, slice_img in enumerate(slices):
    combined_image.paste(slice_img, (i * width, 0))
output_path = "{model_name}.png"
combined_image.save(output_path)
combined_image.show()

# Load Model and Run Inference

In [17]:
with open('Models/{Model Name}.pkl', 'rb') as file:
    model = pickle.load(file)

### Logo Extraction and Enhancement

In [18]:
def sharpen_image(image):
    sharpening_kernel = np.array([[0, -1, 0],
                                   [-1, 5, -1],
                                   [0, -1, 0]])
    sharpened_image = cv2.filter2D(image, -1, sharpening_kernel)
    return sharpened_image

def extract_and_crop_top_n_contours(image_path, n=3, target_size=(140, 140)):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    edges = cv2.Canny(blurred_image, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    cropped_images = []

    for i in range(min(n, len(sorted_contours))):
        x, y, w, h = cv2.boundingRect(sorted_contours[i])
        cropped_img = image[y:y+h, x:x+w]
        resized_img = cv2.resize(cropped_img, target_size)
        sharpened_img = sharpen_image(resized_img)
        cropped_images.append(sharpened_img)
        output_path = f'cropped_logo_{i + 1}.jpg'
        cv2.imwrite(output_path, sharpened_img)
        print(f'Saved: {output_path}')

    return cropped_images
cropped_logos = extract_and_crop_top_n_contours(image_path, n=1)
for i, logo in enumerate(cropped_logos):
    cv2.imshow(f'Cropped Logo {i + 1}', logo)

cv2.waitKey(0)
cv2.destroyAllWindows()


Saved: cropped_logo_1.jpg


### Image processing 

In [2]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (140, 140))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [ ]:
def classify_image(model, image_path):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    predicted_class = np.argmax(predictions, axis=1)
    
    return predicted_class

### Prediction 

In [37]:
image_path = '{Test Image Path}'
predicted_class = classify_image(model, image_path)

print(f'Predicted class: {predicted_class}')


1/1 [==============================] - 1s 802ms/step
Predicted class: [11]
